# NLP Augmentation (Offline)

**In computer vision problems, there is a virtual infinitude of techniques you can use to augment your images ranging from simple techniques like randomly flipping images to blending images together with CutMix or MixUp. In natural language processing, it is not as easy to come up with similar augmentation strategies; we must be a little more creative**

**The first idea I had was to randomly replace words with their synonyms or to randomly add word synonyms to the sequence, but then I saw [this kernel](https://www.kaggle.com/jpmiller/augmenting-data-with-translations) which is based on [this discussion thread](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/48038) and realized we can do better: we can use translation to augment our data and do several things:**

1. We can experiment and see if training our model on one language is better/worse than training on multiple languages
2. We can change the distribution of languages in our dataset, perhaps translating sentences to low-resource languages like Swahili and Urdu
3. We can randomly translate sentences to another language and then translate them back to the original like so:

![](https://amitness.com/images/backtranslation-en-fr.png)

*Image from [@amitness](https://www.kaggle.com/amitness)*

**We can also apply this augmentation in two ways: offline augmentation or online augmentation. In the first, we augment before we feed to the model, adding to our dataset size. This is preferable for smaller datasets where we are not worried about training taking too long. When you can't afford an increase in size, you resort to online augmentation where augment the data every epoch. We will use offline augmentation in this commit (for more [see](https://www.kaggle.com/c/datasciencebowl/discussion/12597))**

**For now, we will only translate to languages currently present in our dataset, but translating to languages outside of our dataset might give us better performance. Please note that some of these language codes are slightly different within the `googletrans` Python API. See [here](https://py-googletrans.readthedocs.io/en/latest/) for more**

In [ ]:
GEN_BACK_TR = True

GEN_UPSAMPLE = False

GEN_EN_ONLY = False

In [ ]:
#python basics
from matplotlib import pyplot as plt
import math, os, re, time
import numpy as np, pandas as pd, seaborn as sns

#nlp augmentation
!pip install --quiet googletrans
from googletrans import Translator

#model evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold

#for fast parallel processing
from dask import bag, diagnostics

In [ ]:
def back_translate(sequence, PROB = 1):
    languages = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi',
                 'sw', 'vi', 'es', 'el']
    
    #instantiate translator
    translator = Translator()
    
    #store original language so we can convert back
    org_lang = translator.detect(sequence).lang
    
    #randomly choose language to translate sequence to  
    random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])
    
    if org_lang in languages:
        #translate to new language and back to original
        translated = translator.translate(sequence, dest = random_lang).text
        #translate back to original language
        translated_back = translator.translate(translated, dest = org_lang).text
    
        #apply with certain probability
        if np.random.uniform(0, 1) <= PROB:
            output_sequence = translated_back
        else:
            output_sequence = sequence
            
    #if detected language not in our list of languages, do nothing
    else:
        output_sequence = sequence
    
    return output_sequence

#check performance
for i in range(5):
    output = back_translate('I genuinely have no idea what the output of this sequence of words will be')
    print(output)

In [ ]:
#applies above define function with Dask
def back_translate_parallel(dataset):
    prem_bag = bag.from_sequence(dataset['premise'].tolist()).map(back_translate)
    hyp_bag =  bag.from_sequence(dataset['hypothesis'].tolist()).map(back_translate)
    
    with diagnostics.ProgressBar():
        prems = prem_bag.compute()
        hyps = hyp_bag.compute()

    #pair premises and hypothesis
    dataset[['premise', 'hypothesis']] = list(zip(prems, hyps))
    
    return dataset

In [ ]:
twice_train_aug = pd.read_csv('../input/contradictorywatsontwicetranslatedaug/twice_translated_aug_train.csv')
twice_test_aug = pd.read_csv('../input/contradictorywatsontwicetranslatedaug/twice_translated_aug_test.csv')

In [ ]:
if GEN_BACK_TR:
#now we apply translation augmentation
    train_thrice_aug = twice_train_aug.pipe(back_translate_parallel)
    test_thrice_aug = twice_test_aug.pipe(back_translate_parallel)
    
    train_thrice_aug.to_csv('thrice_translation_aug_train.csv')
    test_thrice_aug.to_csv('thrice_translation_aug_test.csv')

# For TTA

**I have already created datasets where each premise/hypothesis is separately mapped to a random language and back: these datasets can be found [here](https://www.kaggle.com/tuckerarrants/contradictorywatsontranslationaug) and [here](https://www.kaggle.com/tuckerarrants/contradictorywatsontwicetranslatedaug). One is translated a single time and the other is twice translated:**

**Note that we can augment the test set as well and use test time augmentation (TTA) where we make separate predictions on the original sequences and the augmented sequences, and then use the average of the predictions for our final predictions:**

In [ ]:
#offline loading
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

train_aug = pd.read_csv("../input/contradictorywatsontwicetranslatedaug/translation_aug_train.csv")
test_aug = pd.read_csv("../input/contradictorywatsontwicetranslatedaug/translation_aug_test.csv")

train_twice_aug = pd.read_csv("../input/contradictorywatsontwicetranslatedaug/twice_translated_aug_train.csv")
test_twice_aug = pd.read_csv("../input/contradictorywatsontwicetranslatedaug/twice_translated_aug_test.csv")

#view original
print(train.shape)
train.head()

In [ ]:
#view first aug
print(train_aug.shape)
train_aug.head()

In [ ]:
#view second aug
print(train_twice_aug.shape)
train_twice_aug.head()

In [ ]:
#view third aug
print(train_thrice_aug.shape)
train_thrice_aug.head()

**Wonderful! We can see that this translation procedure consistently alters our sentences without much information loss. Feel free to use these datasets for your own experiments and see how they improve your model's performance**

# For Oversampling

**Now we will use the same idea to create a training dataset of only undersampled languages, like Urdu and Swahili. The only difference here is that we are not translating to random languages or translating back to the original sentence:**

In [ ]:
#check most undersampled languages in training dataset
train['language'].value_counts()

In [ ]:
#check most undersampled languages in test dataset
test['language'].value_counts()

**We have a choose to make here: do we translate based on language prevalence in the training, testing dataset, or based on the languages that XLM-R was trained on? I will base my translation on the test set languages, so I will create a Vietnamese, Hindi, and Bulgarian training datasets, for now:**

In [ ]:
def translation(sequence, lang):
    
    #instantiate translator
    translator = Translator()
    
    org_lang = translator.detect(sequence).lang
    
    if lang is not org_lang:
        #translate to new language and back to original
        translated = translator.translate(sequence, dest = lang).text
        
    else:
        translated = sequence
    
    return translated

def translation_parallel(dataset, lang):
    prem_bag = bag.from_sequence(dataset['premise'].tolist()).map(lambda x: translation(x, lang = lang))
    hyp_bag =  bag.from_sequence(dataset['hypothesis'].tolist()).map(lambda x: translation(x, lang = lang))
    
    with diagnostics.ProgressBar():
        prems = prem_bag.compute()
        hyps = hyp_bag.compute()

    #pair premises and hypothesis
    dataset[['premise', 'hypothesis']] = list(zip(prems, hyps))
    
    return dataset

In [ ]:
#translate to Vietnamese
prem_bag_vi = bag.from_sequence(train['premise'].tolist()).map(lambda x: translation(x, lang = 'vi'))
hyp_bag_vi =  bag.from_sequence(train['hypothesis'].tolist()).map(lambda x: translation(x, lang = 'vi'))

#translate to Hindi
prem_bag_hi = bag.from_sequence(train['premise'].tolist()).map(lambda x: translation(x, lang = 'hi'))
hyp_bag_hi =  bag.from_sequence(train['hypothesis'].tolist()).map(lambda x: translation(x, lang = 'hi'))

#translate to Bulgarian
prem_bag_bg = bag.from_sequence(train['premise'].tolist()).map(lambda x: translation(x, lang = 'bg'))
hyp_bag_bg =  bag.from_sequence(train['hypothesis'].tolist()).map(lambda x: translation(x, lang = 'bg'))

#and compute
if GEN_UPSAMPLE:
    with diagnostics.ProgressBar():
        print('Translating train to Vietnamese...')
        prems_vi = prem_bag_vi.compute()
        hyps_vi = hyp_bag_vi.compute()
        print('Done'); print('')
    
        print('Translating train to Hindi...')
        prems_hi = prem_bag_hi.compute()
        hyps_hi = hyp_bag_hi.compute()
        print('Done'); print('')
    
        print('Translating train to Bulgarian...')
        prems_bg = prem_bag_bg.compute()
        hyps_bg = hyp_bag_bg.compute()
        print('Done')
        
else:
    train_vi = pd.read_csv("../input/contradictorytranslatedtrain/train_vi.csv")
    train_hi = pd.read_csv("../input/contradictorytranslatedtrain/train_hi.csv")
    train_bg = pd.read_csv("../input/contradictorytranslatedtrain/train_bg.csv")

In [ ]:
if GEN_UPSAMPLE:
    #sanity check
    train_vi = train
    train_vi[['premise', 'hypothesis']] = list(zip(prems_vi, hyps_vi))
    train_vi[['lang_abv', 'language']] = ['vi', 'Vietnamese']
    train_vi.to_csv('train_vi.csv', index = False)
train_vi.head()

In [ ]:
if GEN_UPSAMPLE:
    #sanity check
    train_hi = train
    train_hi[['premise', 'hypothesis']] = list(zip(prems_hi, hyps_hi))
    train_hi[['lang_abv', 'language']] = ['hi', 'Hindi']
    train_hi.to_csv('train_hi.csv', index = False)
train_hi.head()

In [ ]:
if GEN_UPSAMPLE:
    #sanity check
    train_bg = train
    train_bg[['premise', 'hypothesis']] = list(zip(prems_bg, hyps_bg))
    train_bg[['lang_abv', 'language']] = ['bg', 'Bulgarian']
    train_bg.to_csv('train_bg.csv', index = False)
train_bg.head()

# English Only

In [ ]:
#translate to English
prem_bag_en = bag.from_sequence(train['premise'].tolist()).map(lambda x: translation(x, lang = 'en'))
hyp_bag_en =  bag.from_sequence(train['hypothesis'].tolist()).map(lambda x: translation(x, lang = 'en'))

if GEN_EN_ONLY:
    #sanity check
    train_en = train
    train_en[['premise', 'hypothesis']] = list(zip(prems_en, hyps_en))
    train_en[['lang_abv', 'language']] = ['en', 'English']
    train_en.to_csv('train_en.csv', index = False)

else:
    train_en = pd.read_csv("../input/contradictorytranslatedtrain/train_en.csv")
    
#sanity check
train_en.head()

**The English translated dataset can now be found [here](https://www.kaggle.com/tuckerarrants/contradictorytranslatedtrain)**